In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from simulation import Simulation,TreeSimilarity
from test_simulation import *

In [3]:
var={'n':5,
     'p_otu':8, 
     'p_metabolite':4}

In [4]:
s = Simulation(**var)

In [5]:
# otu table, metabolite table run on default params
s.run_simulation()

(array([[-1.94582134e+00, -4.07855232e+00, -3.69175682e+00,
         -4.65928599e+00, -1.44152077e+00],
        [ 5.81005242e-01,  5.04609712e-01,  8.66643263e-01,
          2.26119804e-02,  2.84016487e-03],
        [ 5.19911782e-01,  9.90337173e-01,  4.69284641e-01,
          4.11319284e-01,  9.77673340e-01],
        [ 4.91906573e-01,  9.06910576e-01,  4.48510121e-01,
          8.23557483e-01,  9.04422882e-01],
        [ 4.40495166e-01,  3.12096336e-01,  5.92939414e-01,
          8.17613083e-01,  4.04801921e-01],
        [ 8.07426190e-01,  3.48753402e-01,  1.59865950e-01,
          7.54712532e-01,  6.71164521e-01],
        [ 3.14664402e-01,  9.46494843e-01,  9.98218247e-01,
          9.20297579e-01,  3.53647870e-01],
        [ 4.94947043e-01,  8.56819525e-01,  4.62993260e-01,
          6.84894766e-01,  2.05865047e-01]]),
 array([[0.35464056, 1.40765647, 1.27362692, 1.13956932, 0.10538722],
        [0.13717889, 0.79512513, 0.06787464, 0.81764162, 0.61511892],
        [0.72492778, 0.969

# multivariate normal dist

In [48]:
otu_t=TreeSimilarity(var["p_otu"])
met_t=TreeSimilarity(var["p_metabolite"],names="metabolite")

In [49]:
display(otu_t.draw())
display(met_t.draw())

  ____________________________________________ otu2
 |
 |                                                         ______________ otu6
 |                                        ________________|
_|                                       |                |______________ otu3
 |                      _________________|
 |                     |                 |                 ______________ otu7
 |                     |                 |         _______|
 |                     |                 |________|       |______________ otu5
 |_____________________|                          |
                       |                          |_____________________ otu1
                       |
                       |                          ______________ otu4
                       |_________________________|
                                                 |______________ otu0



None

                                      _____________________________ metabolite3
  ___________________________________|
 |                                   |_____________________________ metabolite1
_|
 |                                          _______________________ metabolite2
 |_________________________________________|
                                           |_______________________ metabolite0



None

In [54]:
var2={'n':5,
     'p_otu':8, 
     'p_metabolite':4,
     'multivar':True,
     'A':otu_t.cov,
     'B':met_t.cov}

s2=Simulation(**var2)

In [56]:
display(s2.otutable,s2.metabolitetable)

array([[ 0.56343746, -0.03058826, -0.55765963,  0.24817555, -0.22894977,
         0.56790744, -0.15042191,  0.51329601],
       [-0.32510871,  0.08601607,  0.07830876,  0.20279903, -0.03579529,
         0.28306919,  0.14639478,  0.06832128],
       [ 0.16140752, -0.08565895,  0.01253758,  0.02932415,  0.25078535,
        -0.27864089,  0.06725617, -0.25883035],
       [ 0.47775402, -0.18525365, -0.11158259, -0.14541984, -0.0631256 ,
        -0.12669395, -0.10312233, -0.07828237],
       [ 0.0874227 ,  0.02285167, -0.09857239,  0.0459499 ,  0.41264204,
        -0.07546229, -0.29099246, -0.04095715]])

array([[ 0.47021681, -0.58912602,  0.62308918, -0.14416621],
       [ 0.24361391, -0.47260287,  0.6247049 , -0.43181518],
       [-0.15980325, -0.01916658,  0.21253678, -0.45827424],
       [ 0.17530845, -0.53548776,  0.62247496, -0.48178934],
       [-0.12321298,  0.02081315,  0.07156665, -0.23889863]])